In [ ]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

# Classification

Loading data

In [ ]:
# loading data
data = pd.read_csv('../data/microchip_tests.txt',
                   header=None, names = ('test1','test2','released'))
# info about dataframe
data.info()

In this dataset on 118 microchips (objects), there are results for two tests of quality control (two numerical variables) and information whether the microchip went into production. Variables are already centered, meaning that the column values have had their own mean values subtracted. Thus, the "average" microchip corresponds to a zero value in the test results.

Let's inspect at the first and last 5 lines.

In [ ]:
data.head(5)

In [ ]:
data.head(5)

Data manipulation with Pandas

In [ ]:
data.loc[1:10, 'released']

In [ ]:
data.iloc[[0, 3], :]

Put training data and class labels into separate numpy arrays

In [ ]:
X = data.iloc[:,:2].values
y = data.iloc[:,2].values

As an intermediate step, we can plot the data. Orange points correspond to defective chips, blue to normal ones.

In [ ]:
plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Released')
plt.scatter(X[y == 0, 0], X[y == 0, 1], c='orange', label='Faulty')
plt.xlabel("Test 1")
plt.ylabel("Test 2")
plt.title('2 tests of microchips. Logit with C=1')
plt.legend();

Let's define a function to display the separating curve of the classifier.

In [ ]:
def plot_boundary(clf, X, y, grid_step=.01, poly_featurizer=None):
    x_min, x_max = X[:, 0].min() - .1, X[:, 0].max() + .1
    y_min, y_max = X[:, 1].min() - .1, X[:, 1].max() + .1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, grid_step),
                         np.arange(y_min, y_max, grid_step))


    # to every point from [x_min, m_max]x[y_min, y_max]
    # we put in correspondence its own color
    if poly_featurizer == None:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = clf.predict(poly_featurizer.transform(np.c_[xx.ravel(), yy.ravel()]))
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z, cmap=plt.cm.Paired)

Split our data into train and test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Let's train logistic regression with regularization parameter $C = 1$- without regularization.

In [ ]:
C=1

We will use `sklearn`'s implementation of logistic regression.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logit = LogisticRegression(random_state=17)

In [ ]:
logit.fit(X_train, y_train)

In [ ]:
plot_boundary(logit, X, y, grid_step=.01)

plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Released')
plt.scatter(X[y == 0, 0], X[y == 0, 1], c='orange', label='Faulty')
plt.xlabel("Test 1")
plt.ylabel("Test 2")
plt.title('2 tests of microchips. Logit with C=%s' % C)
plt.legend();

print("Accuracy on training set:", 
      round(logit.score(X_train, y_train), 3))

print("Accuracy on testing set:", 
      round(logit.score(X_test, y_test), 3))

Let's calculate and plot ROC AUC

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
def roc(clf, X_test):
    probas_ = logit.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2)
    plt.title('ROC AUC = %0.2f' % roc_auc)

In [ ]:
roc(logit, X_test)

As we can see our classifier is not very good since the bounday is nonlinear.

So, we create an object that will add polynomial features up to degree 7 to matrix $X$.

We define the following polynomial features of degree $d$ for two variables $x_1$ and $x_2$:

$$\large \{x_1^d, x_1^{d-1}x_2, \ldots x_2^d\} =  \{x_1^ix_2^j\}_{i+j=d, i,j \in \mathbb{N}}$$

For example, for $d=3$, this will be the following features:

$$\large 1, x_1, x_2,  x_1^2, x_1x_2, x_2^2, x_1^3, x_1^2x_2, x_1x_2^2, x_2^3$$


In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=7)
X_poly = poly.fit_transform(X)

In [ ]:
X_poly.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

In [ ]:
C = 1e-2
logit = LogisticRegression(C=C, random_state=17)
logit.fit(X_train, y_train)

In [ ]:
plot_boundary(logit, X, y, grid_step=.01, poly_featurizer=poly)

plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Released')
plt.scatter(X[y == 0, 0], X[y == 0, 1], c='orange', label='Faulty')
plt.xlabel("Test 1")
plt.ylabel("Test 2")
plt.title('2 tests of microchips. Logit with C=%s' % C)
plt.legend();

print("Accuracy on training set:", 
      round(logit.score(X_train, y_train), 3))

print("Accuracy on testing set:", 
      round(logit.score(X_test, y_test), 3))

In [ ]:
roc(logit, X_test)

We could now try increasing $C$ to 1. In doing this, we weaken regularization, and the solution can now have greater values (in absolute value) of model weights than previously. Now the accuracy of the classifier on the training set improves to 0.831.

In [ ]:
C = 1
logit = LogisticRegression(C=C, random_state=17)
logit.fit(X_train, y_train)

plot_boundary(logit, X, y, grid_step=.005, poly_featurizer=poly)

plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Released')
plt.scatter(X[y == 0, 0], X[y == 0, 1], c='orange', label='Faulty')
plt.xlabel("Test 1")
plt.ylabel("Test 2")
plt.title('2 tests of microchips. Logit with C=%s' % C)
plt.legend();

print("Accuracy on training set:", 
      round(logit.score(X_train, y_train), 3))

print("Accuracy on testing set:", 
      round(logit.score(X_test, y_test), 3))

In [ ]:
roc(logit, X_test)

Then, why don't we increase $C$ even more - up to 10,000? Now, regularization is clearly not strong enough, and we see overfitting. Note that, with $C$=1 and a "smooth" boundary, the share of correct answers on the training set is not much lower than here. But one can easily imagine how our second model will work much better on new data.

In [ ]:
C = 1e4
logit = LogisticRegression(C=C, random_state=17)
logit.fit(X_train, y_train)

plot_boundary(logit, X, y, grid_step=.005, poly_featurizer=poly)

plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Released')
plt.scatter(X[y == 0, 0], X[y == 0, 1], c='orange', label='Faulty')
plt.xlabel("Test 1")
plt.ylabel("Test 2")
plt.title('2 tests of microchips. Logit with C=%s' % C)
plt.legend();

print("Accuracy on training set:", 
      round(logit.score(X_train, y_train), 3))

print("Accuracy on testing set:", 
      round(logit.score(X_test, y_test), 3))

In [ ]:
roc(logit, X_test)

To discuss the results, let's rewrite the function that is optimized in logistic regression with the form:

$$\large J(X,y,w) = \mathcal{L} + \frac{1}{C}||w||^2,$$

where

- $\mathcal{L}$ is the logistic loss function summed over the entire dataset
- $C$ is the reverse regularization coefficient (the very same $C$ from `sklearn`'s implementation of `LogisticRegression`)

**Subtotals**:
- the larger the parameter $C$, the more complex the relationships in the data that the model can recover (intuitively $C$ corresponds to the "complexity" of the model - model capacity)
- if regularization is too strong i.e. the values of $C$ are small, the solution to the problem of minimizing the logistic loss function may be the one where many of the weights are too small or zeroed. The model is also not sufficiently "penalized" for errors (i.e. in the function $J$, the sum of the squares of the weights "outweighs", and the error $\mathcal{L}$ can be relatively large). In this case, the model will underfit as we saw in our first case.
- on the contrary, if regularization is too weak i.e. the values of $C$ are large, a vector $w$ with high absolute value components can become the solution to the optimization problem. In this case, $\mathcal{L}$ has a greater contribution to the optimized functional $J$. Loosely speaking, the model is too "afraid" to be mistaken on the objects from the training set and will therefore overfit as we saw in the third case.
- logistic regression will not "understand" (or "learn") what value of $C$ to choose as it does with the weights $w$. That is to say, it can not be determined by solving the optimization problem in logistic regression. We have seen a similar situation before -- a decision tree can not "learn" what depth limit to choose during the training process. Therefore, $C$ is the a model hyperparameter that is tuned on cross-validation; so is the max_depth in a tree.

# More complicated data

Let's work with dataset where given various features of the flight we are asked to predict wheather there will be a delay.

In [ ]:
data = pd.read_csv("../data/flight_delays.csv")

Split features and target

In [ ]:
X, y = data.iloc[:, :-1], data.iloc[:, -1]

Since we have a lot of categorical features it is a good idea to onehotencode them

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X.head()

We also need to encode our labels to be 0 or 1

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
coder = LabelEncoder()

In [ ]:
coder.fit(y)

In [ ]:
y = coder.transform(y)

# Your task

Fit `sklearn` implementation of `SGDClassifier` (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn-linear-model-sgdclassifier). You also have to split dataset appropriately to the train and test sets and plot ROC curve and AUC. 

# Bonus task

Use cross-validation to choose appropriate constant for regularization coefficient $C$ for $L2$ regularizer (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)